# Aligning data — example notebook

In [ ]:
from imports import *
from load_data_functions import *

---
## Reading in IRIS and AIA data as SunPy maps

In [ ]:
# Reading in AIA 171 data -- USING DATA DOWNLOADED FROM JSOC
folder = 'ex_data/JSOC_20221206_1621' # path to folder with AIA data
aia171_maps = get_aia_data(folder)

In [ ]:
# Reading in IRIS 1400 data -- USING DATA DOWNLOADED FROM IRIS WEBSITE
filename = 'ex_data/iris_l2_20220305_150930_3600607418_SJI_1400_t000.fits' # path to fits with IRIS data
iris1400_maps = get_iris_data(filename)

---
## Cut observations so they cover the same time

In [ ]:
# get overlap times between AIA and IRIS observations
start_time, end_time = obs_overlap_times(aia171_maps, iris1400_maps)
print(start_time, end_time)

In [ ]:
aia171_maps_cut = np.array(cut_obs(aia171_maps, start_time, end_time))
iris1400_maps_cut = np.array(cut_obs(iris1400_maps, start_time, end_time))

---
## Matching observations 

AIA and IRIS satellites observe at difference cadences. To account for this, "match_obs" matches each image of the AIA observation with an image from the IRIS observation that is the closest in time.

This makes both sets of observations contain the same number of images.

(If these two instruments were at different distances from Earth, "match_obs" would account for that and match images with respective to Earth time.)

In [ ]:
# MATCHING OBSERVATIONS TO EUI -- MAKE ALL SAME LENGTH
aia171_maps_cut, sort_i = sort_data(aia171_maps_cut)

times_to_match = get_times(aia171_maps_cut)
times_to_match = get_earth_times(aia171_maps_cut) # get earth times

iris1400_matched = match_obs(iris1400_maps_cut, times_to_match)
aia171_matched = match_obs(aia171_maps_cut, times_to_match)

---

## Correct for solar rotation

In [ ]:
# already corrected for in IRIS -- rotation already corrected for in data downloaded from IRIS website

aia171_rot = correct_diff_rot_obs(aia171_matched, ref_i=0) # align to first observation in the array

---

## Align all observations to the same reference frame
aligning to the middle AIA observation

In [ ]:
# ALIGN ALL OBS TO THE SAME REFERENCE FRAME -- AIA MIDDLE OBS
ref_obs = aia171_matched[len(aia171_matched)//2] # middle AIA observation

In [ ]:
aia171_aligned = align_obs_to_reference(aia171_rot, ref_obs)

iris1400_aligned = align_obs_to_reference(iris1400_matched, ref_obs)

---
## Plotting IRIS 1400 and AIA 171

In [ ]:
# PLOT 4 DATASETS
i = 100

map1 = iris1400_aligned[i] # iris
map1_time = iris1400_matched[i].date # need to access the non-aligned metadata for the correct time
map1_name = 'IRIS 1400'
map1_cmap = map1.cmap
map1_norm = plt.Normalize(-5, 15)

map2 = aia171_aligned[i] # aia 171
map2_time = aia171_matched[i].date
map2_name = 'AIA 171'
map2_cmap = map2.cmap
map2_norm = plt.Normalize(0, 2700)


####

%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 5]
fig, axs = plt.subplots(1, 2)

axs[0].axis('off'); axs[1].axis('off')

# map1 -- IRIS 1400
ax1 = fig.add_subplot(121, projection=map1.wcs)
ax1.imshow(map1.data, cmap=map1_cmap, norm=map1_norm)
x, y = map1.data.shape
ax1.set_title(f'{map1_name}\n{map1_time.datetime.time()}', fontsize=8)

# map2 -- AIA 171
ax2 = fig.add_subplot(122, projection=map1.wcs, sharex=ax1, sharey=ax1)
ax2.imshow(map2.data, cmap=map2_cmap, norm=map2_norm)
x, y = map2.data.shape
ax2.set_title(f'{map2_name}\n{map2_time.datetime.time()}', fontsize=8)

plt.tight_layout(pad = 4)
plt.show()